# ПЗ 5

## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации  
2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки  
3. вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)  
4. взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)    
5. сравнить все реализованные методы сделать выводы

### загрузка данных

In [1]:
!pip install pyconll

In [2]:
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger
import numpy as np
from scipy.sparse import hstack
import pyconll

In [3]:
!mkdir datasets

In [4]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-07-07 16:08:52--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   170MB/s    in 0.5s    

2021-07-07 16:08:56 (170 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-07-07 16:08:56--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [5]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [6]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [7]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_sent_train = []
for sent in full_train[:]:
    fdata_sent_train.append([token.form.lower() for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form.lower() for token in sent])

### 1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации 

In [8]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.8819634010716814

In [9]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.8882907019849695

In [10]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.8874566103865467

In [11]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_test,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.983924780103124

Посмотрим распределение тегов

In [12]:
fdata_pairs_test = []
for sent in fdata_test[:]:
    fdata_pairs_test.extend(i for i in sent)

tags = [tag for (word, tag) in fdata_pairs_test]

In [13]:
nltk.FreqDist(tags)

FreqDist({'ADJ': 11222,
          'ADP': 10585,
          'ADV': 6165,
          'AUX': 1108,
          'CCONJ': 4410,
          'DET': 3085,
          'INTJ': 11,
          'NOUN': 27974,
          'NUM': 1829,
          None: 204,
          'PART': 3875,
          'PRON': 5598,
          'PROPN': 4438,
          'PUNCT': 22694,
          'SCONJ': 2258,
          'SYM': 53,
          'VERB': 13078,
          'X': 105})

### 2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки

In [14]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [16]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [17]:
vectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='char', max_features=30)

In [18]:
X_train = vectorizer.fit_transform(train_tok)
X_test = vectorizer.transform(test_tok)

In [19]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
pred = lr.predict(X_test)

In [21]:
accuracy_score(test_enc_labels, pred)

0.6128972466552084

<b><font color='blue'>
    0.612 - скор для бейзлайна (всего 30 признаков)
</font><b>

In [22]:
word_vectorizer = CountVectorizer(ngram_range=(1,5), analyzer='word', max_features=30)

In [23]:
X_train_word = word_vectorizer.fit_transform(train_tok)
X_test_word = word_vectorizer.transform(test_tok)

In [24]:
lr1 = LogisticRegression(random_state=0, max_iter=10)
lr1.fit(X_train_word, train_enc_labels)
pred1 = lr1.predict(X_test_word)
accuracy_score(test_enc_labels, pred1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.3118070299598962

<b><font color='blue'>
    анализировать только по словам не очень, попробуем объединить два подхода <br>
    При использовании только vectorizer = CountVectorizer(ngram_range=(1, 9), analyzer='char') скор 0.79
</font><b>

In [25]:
vectorizer = CountVectorizer(ngram_range=(1, 9), analyzer='char')
X_train_c = vectorizer.fit_transform(train_tok)
X_test_c = vectorizer.transform(test_tok)

In [26]:
word_vectorizer = CountVectorizer(ngram_range=(1,5), analyzer='word')
X_train_w = word_vectorizer.fit_transform(train_tok)
X_test_w = word_vectorizer.transform(test_tok)

In [27]:
X_train = hstack((X_train_c, X_train_w))
X_test = hstack((X_test_c, X_test_w))

In [28]:
X_train.shape, X_train_c.shape, X_train_w.shape

((871526, 692910), (871526, 593425), (871526, 99485))

In [29]:
lr = LogisticRegression(random_state=0, max_iter=25)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9121255012974758

### 3. вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)  

In [30]:
import gensim

In [31]:
train_word2vec = fdata_sent_train
train_word2vec.extend(fdata_sent_test)

In [32]:
w2v_model = gensim.models.Word2Vec(min_count=1, negative=20, sg=1, window=2)
w2v_model.build_vocab(train_word2vec)
w2v_model.train(train_word2vec, epochs=5, total_examples=w2v_model.corpus_count)

(3849944, 4951090)

In [33]:
X_train_w2v = [w2v_model.wv[word] for word in train_tok]
X_test_w2v = [w2v_model.wv[word] for word in test_tok]

In [34]:
lr = LogisticRegression(random_state=0, max_iter=25)
lr.fit(X_train_w2v, train_enc_labels)
pred = lr.predict(X_test_w2v)
accuracy_score(test_enc_labels, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7361742998685674

<b><font color='blue'>
0.74 приемлимый скор для минималистичной модели
</font><b>

In [35]:
ft_model = gensim.models.FastText(train_word2vec, min_count=1, negative=20, sg=1, window=2)

In [36]:
X_train_ft = [ft_model.wv[word] for word in train_tok]
X_test_ft = [ft_model.wv[word] for word in test_tok]

In [37]:
lr = LogisticRegression(random_state=0, max_iter=25)
lr.fit(X_train_ft, train_enc_labels)
pred = lr.predict(X_test_ft)
accuracy_score(test_enc_labels, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8640430694570822

### 4. взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)   

In [38]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
windows = [1,2,3,4,5,6,7,8,9]
for i in windows:
    ft_model = gensim.models.FastText(train_word2vec, min_count=1, negative=20, sg=1, window=i)
    X_train_ft = [ft_model.wv[word] for word in train_tok]
    X_test_ft = [ft_model.wv[word] for word in test_tok]
    lr.fit(X_train_ft, train_enc_labels)
    pred = lr.predict(X_test_ft)
    print(i, accuracy_score(test_enc_labels, pred))

1 0.8614565429852054
2 0.8621642570687157
3 0.8576989182084723
4 0.8593839517406396
5 0.855247194419169
6 0.8519698041991035
7 0.8528628719711522
8 0.8528038957975264
9 0.8513884676305058


### 5. сравнить все реализованные методы сделать выводы

<b><font color='blue'>
Лучше всего отработали комбинации теггеров. <br>
На самом деле все подходы можно считать работающими, просто результаты получились не идеальными, потому что для обучения мы использовали небольшой корпус.
</font><b>

## Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [40]:
!pip install corus

     |████████████████████████████████| 92kB 5.2MB/s 


In [41]:
import corus
import pandas as pd

### Прочтем данные

In [42]:
 !wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-07-07 16:44:56--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   362KB/s    in 5.9s    

2021-07-07 16:45:02 (315 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [43]:
 !unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [44]:
from corus import load_ne5

path = 'Collection5/'
records = load_ne5(path)
next(records)

Ne5Markup(
    id='25_12_12d',
    text='"Салават Юлаев" уволил главного тренера\r\nУфимский "Салават Юлаев" отправил в отставку главного тренера Венера Сафина. Об этом сообщает официальный сайт команды. О причинах увольнения тренера ничего не сообщается.\r\n\r\nИсполняющим обязанности наставника "Салавата Юлаева" назначен Владимир Юрзинов, ранее работавший помощником Сафина. Другой его помощник Василий Чижов также остался в тренерском штабе.\r\n\r\nСафин, работавший в "Салавате" с 2001 года, возглавил уфимский клуб в ноябре 2011 года после увольнения Сергея Михалева. В текущем сезоне "Салават Юлаев" с 58-ю очками после 37 игр занимает пятое место в таблице Восточной конференции. В последней игре уфимцы в гостях уступили московскому ЦСКА со счетом 1:4.\r\n\r\nСафин стал 11-м тренером, уволенным с начала сезона КХЛ. Ранее работы лишились Кари Хейккиля в минском "Динамо", Янне Карлссон в мытищинском "Атланте", Йозеф Яндач в пражском "Льве", Андрей Шаянов в екатеринбургском "Автомобилисте

### 1. Воспользуемся NER из nltk

In [45]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [46]:
document = next(records).text

In [47]:
document

'Топ-менеджер Bloomberg возглавит агентство Dow Jones\r\n\r\nОдин из топ-менеджеров компании Bloomberg Лекс Фенуик (Lex Fenwick) возглавит с 13 февраля агентство финансовых новостей Dow Jones & Company. Об этом сообщает Reuters.\r\n\r\nФенуик проработал в компании Bloomberg, также занимающейся финансовыми новостями, 25 лет. В 2001 году Фенуик сменил Майкла Блумберга (Michael Bloomberg) на должности главы компании. В 2008 Фенуик основал подразделение Bloomberg Ventures, которое и возглавлял в последние годы.\r\n\r\nБывший глава Dow Jones Лес Хинтон (Les Hinton) оставил свою должность в июле 2011 года после скандала с прослушками в таблоиде News of The World. До 2007 года Хинтон возглавлял News International, британское подразделение принадлежащей Руперту Мердоку компании News Corp. Компания News International издавала до июля 2011 года таблоид News of The World, закрывшийся из-за обвинений в прослушке. Большая часть инцидентов с прослушкой пришлась как раз на тот период, когда Хинтон во

In [48]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Bloomberg', 'ORGANIZATION'),
 ('Company', 'ORGANIZATION'),
 ('Dow Jones Лес Хинтон', 'PERSON'),
 ('Jones', 'PERSON'),
 ('Jones Один', 'PERSON'),
 ('Les Hinton', 'PERSON'),
 ('Lex Fenwick', 'ORGANIZATION'),
 ('Michael Bloomberg', 'PERSON'),
 ('New York Times', 'GPE'),
 ('Sun', 'ORGANIZATION'),
 ('Times', 'ORGANIZATION'),
 ('Times Newspapers Ltd', 'ORGANIZATION'),
 ('Todd Larsen', 'PERSON'),
 ('Wall Street Journal', 'FACILITY'),
 ('Джеймс Мердок', 'PERSON'),
 ('Компания', 'PERSON'),
 ('Майкла Блумберга', 'PERSON'),
 ('Помимо', 'PERSON'),
 ('Помимо Хинтона', 'PERSON'),
 ('Ребека Брукс', 'PERSON'),
 ('Руперту Мердоку', 'PERSON'),
 ('Фенуик', 'PERSON'),
 ('Хинтон', 'PERSON')}

### 3. Попробуем написать свой нер попробовать разные подходы 

In [49]:
!pip install razdel

In [50]:
from razdel import tokenize

In [51]:
records = corus.load_ne5(path)
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [52]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [53]:
df_words['tag'].value_counts()

None        219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [54]:
df_words.head(5)

,word,tag
0,"""",None
1,Салават,ORG
2,Юлаев,ORG
3,"""",None
4,уволил,None


In [55]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [56]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [57]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [58]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [59]:

def custom_standardization(input_data):
        return input_data

def data_prep(train_data, seq_len=1, vocab_size = 30000):    
    
    vocab_size = 30000
    #seq_len = 1

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [60]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [61]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 228s 18ms/step - loss: 0.2844 - accuracy: 0.9170 - val_loss: 0.2080 - val_accuracy: 0.9391
Epoch 2/5
12444/12444 [==============================] - 241s 19ms/step - loss: 0.1217 - accuracy: 0.9637 - val_loss: 0.2867 - val_accuracy: 0.8948
Epoch 3/5
12444/12444 [==============================] - 229s 18ms/step - loss: 0.1086 - accuracy: 0.9655 - val_loss: 0.2326 - val_accuracy: 0.8934
Epoch 4/5
12444/12444 [==============================] - 238s 19ms/step - loss: 0.1036 - accuracy: 0.9663 - val_loss: 0.2480 - val_accuracy: 0.8927
Epoch 5/5
12444/12444 [==============================] - 237s 19ms/step - loss: 0.1008 - accuracy: 0.9671 - val_loss: 0.2418 - val_accuracy: 0.8928


In [62]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 3, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 231s 18ms/step - loss: 0.2977 - accuracy: 0.9134 - val_loss: 0.2113 - val_accuracy: 0.9386
Epoch 2/5
12444/12444 [==============================] - 231s 19ms/step - loss: 0.1263 - accuracy: 0.9623 - val_loss: 0.2072 - val_accuracy: 0.9417
Epoch 3/5
12444/12444 [==============================] - 233s 19ms/step - loss: 0.1101 - accuracy: 0.9653 - val_loss: 0.2081 - val_accuracy: 0.9417
Epoch 4/5
12444/12444 [==============================] - 237s 19ms/step - loss: 0.1048 - accuracy: 0.9660 - val_loss: 0.2443 - val_accuracy: 0.8941
Epoch 5/5
12444/12444 [==============================] - 234s 19ms/step - loss: 0.1018 - accuracy: 0.9667 - val_loss: 0.2519 - val_accuracy: 0.8928


In [63]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 5, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 221s 18ms/step - loss: 0.2959 - accuracy: 0.9132 - val_loss: 0.2157 - val_accuracy: 0.9384
Epoch 2/5
12444/12444 [==============================] - 228s 18ms/step - loss: 0.1257 - accuracy: 0.9624 - val_loss: 0.4022 - val_accuracy: 0.8946
Epoch 3/5
12444/12444 [==============================] - 234s 19ms/step - loss: 0.1098 - accuracy: 0.9655 - val_loss: 0.2572 - val_accuracy: 0.8937
Epoch 4/5
12444/12444 [==============================] - 233s 19ms/step - loss: 0.1045 - accuracy: 0.9663 - val_loss: 0.2361 - val_accuracy: 0.8911
Epoch 5/5
12444/12444 [==============================] - 223s 18ms/step - loss: 0.1013 - accuracy: 0.9670 - val_loss: 0.2290 - val_accuracy: 0.9375


# Вывод.

<b><font color='blue'>
Длинна последовательности практически не влияет на результат.